#ENSF 612
## Final Project

###Authors:

Evan Boerchers, Do Trong Anh, Hunter Kimmett

In [0]:
%pip install nltk
%pip install bs4
%pip install markdown2
%pip install html2text
%pip install lxml

Python interpreter will be restarted.
Collecting nltk
 Downloading nltk-3.5.zip (1.4 MB)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.7/site-packages (from nltk) (0.14.1)
Collecting click
 Downloading click-7.1.2-py2.py3-none-any.whl (82 kB)
Collecting regex
 Downloading regex-2020.11.13-cp37-cp37m-manylinux2014_x86_64.whl (719 kB)
Collecting tqdm
 Downloading tqdm-4.54.1-py2.py3-none-any.whl (69 kB)
Building wheels for collected packages: nltk
 Building wheel for nltk (setup.py): started
 Building wheel for nltk (setup.py): finished with status 'done'
 Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434675 sha256=7f297d11b41fd1110e3eacaa8ffbf76ce5f910d69afc4ddff5742c1d7e1c22d2
 Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
Installing collected packages: tqdm, regex, click, nltk
Successfully installed click-7.1.2 nltk-3.5 regex-2020.11.13 tqdm-4.54.1
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting bs4
 Downloading bs4-0.0.1.tar.gz (1.1 kB)
Collecting beautifulsoup4
 Downloading beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
Collecting soupsieve>1.2
 Downloading soupsieve-2.1-py3-none-any.whl (32 kB)
Building wheels for collected packages: bs4
 Building wheel for bs4 (setup.py): started
 Building wheel for bs4 (setup.py): finished with status 'done'
 Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=78f37bf48fdc5141634b499f43d1a48bb03330b34a9ef6c555d5cf42a27b8e91
 Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Installing collected packages: soupsieve, beautifulsoup4, bs4
Successfully installed beautifulsoup4-4.9.3 bs4-0.0.1 soupsieve-2.1
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting markdown2
 Downloading markdown2-2.3.10-py2.py3-none-any.whl (34 kB)
Installing collected packages: markdown2
Successfully installed markdown2-2.3.10
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting html2text
 Downloading html2text-2020.1.16-py3-none-any.whl (32 kB)
Installing collected packages: html2text
Successfully installed html2text-2020.1.16
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting lxml
 Downloading lxml-4.6.2-cp37-cp37m-manylinux1_x86_64.whl (5.5 MB)
Installing collected packages: lxml
Successfully installed lxml-4.6.2
Python interpreter will be restarted.

### Data Collection

The aqquired readmes where found using the random readme python script. Using the heading_extractor a csv of all headings was generated.

In [0]:
df_new_dataset_labelled = sqlContext.sql("SELECT * FROM dataset_labelling_full_csv")

### Data Labelling

Each new sample was labelled by 2 people.

In [0]:
df_new_dataset_labelled.show(5)

+----------+-------+--------------------+--------------------+--------------------+-------------+------+------+
section_id|file_id| url| heading_markdown| local_readme_file|heading_level|code_1|code_2|
+----------+-------+--------------------+--------------------+--------------------+-------------+------+------+
 1| 1|https://github.co...|# Acunote's JavaS...|acunote.acunote-s...| 1| 0| 0|
 2| 1|https://github.co...| ## Overview|acunote.acunote-s...| 2| 0| 0|
 3| 1|https://github.co...| ## Try It|acunote.acunote-s...| 2| 1| 1|
 4| 1|https://github.co...| ## Features|acunote.acunote-s...| 2| 0| 0|
 5| 1|https://github.co...|## Four Simple St...|acunote.acunote-s...| 2| 1| 1|
+----------+-------+--------------------+--------------------+--------------------+-------------+------+------+
only showing top 5 rows

#### Percent agreement

In [0]:
def percent_agreement(y1, y2):
  assert(len(y1) == len(y2))
  entries = len(y1)
  same_entries = 0
  for i in range (0, entries):
    if (y1[i] == y2[i]):
      same_entries += 1
  return same_entries / entries

In [0]:
code_1_list = df_new_dataset_labelled.select('code_1').rdd.map(lambda x: x[0]).collect()
code_2_list = df_new_dataset_labelled.select('code_2').rdd.map(lambda x: x[0]).collect()

In [0]:
pa_score = percent_agreement(code_1_list , code_2_list)
print("The percent agreement for our labelling is: {:.2f}".format(pa_score))

The percent agreement for our labelling is: 0.83

#### Cohen Kappa Score

In [0]:
from sklearn.metrics import cohen_kappa_score
import numpy as np

In [0]:
ck_score = cohen_kappa_score(np.array(code_1_list), np.array(code_2_list))
print("The cohen kappa for our labeling is: {:.2f}".format(ck_score))

The cohen kappa for our labeling is: 0.66

### Correcting Labelling Disagreements

To determine the label of contested samples the third labeller acted as the tie breaker. A new csv with the final classification for each sample can then be generated.

#### Finding Disagreements

In order to for the third labeller to correct the disagreements the section_id for contested sections is determined.

In [0]:
contested_sections = df_new_dataset_labelled.where(df_new_dataset_labelled['code_1'] != df_new_dataset_labelled['code_2']).select('section_id').rdd.map(lambda x: x[0]).collect()
print("The contested sections are: ", contested_sections)

The contested sections are: [10, 14, 15, 25, 28, 30, 35, 52, 54, 56, 67, 70, 71, 72, 73, 74, 80, 89, 91, 94, 104, 108, 111, 120, 126, 129, 130, 133, 140, 150, 156, 158, 161, 179, 214, 238, 244, 245, 246, 249, 251, 255, 256, 261, 265, 266, 309, 318, 319, 320, 321, 322, 323, 324, 325, 333, 334, 338, 339, 340, 355, 356, 367, 368, 371, 373, 378, 380, 381, 382, 384, 388, 390, 395, 396, 397, 398, 399, 400, 401, 402, 405, 417, 437, 471, 472, 473, 498]

### Final Fully Labelled Data
The final fully labelled dataset is shown below. Additionally useing the section_extractor algorithm section_contents were extracted for each header.

In [0]:
df_new = sqlContext.sql("SELECT * FROM new_sections_csv")
df_new.show()

+----------+-------+--------------------+--------------------+------------+--------------------+-------------+--------------------+
section_id|file_id| url| heading_markdown|section_code| local_readme_file|heading_level| section_contents|
+----------+-------+--------------------+--------------------+------------+--------------------+-------------+--------------------+
 1| 1|https://github.co...|# Acunote's JavaS...| 0|acunote.acunote-s...| 1| %%NEWLINE %%NEWLINE|
 2| 1|https://github.co...| ## Overview| 0|acunote.acunote-s...| 2|%%NEWLINE Acunote...|
 3| 1|https://github.co...| ## Try It| 1|acunote.acunote-s...| 2|Firefox extension...|
 4| 1|https://github.co...| ## Features| 0|acunote.acunote-s...| 2|%%NEWLINE * Sim...|
 5| 1|https://github.co...|## Four Simple St...| 1|acunote.acunote-s...| 2|%%NEWLINE 1. <a h...|
 6| 1|https://github.co...|## Four Simple St...| 1|acunote.acunote-s...| 2|%%NEWLINE Rails h...|
 7| 1|https://github.co...|## News.YC, Reddi...| 0|acunote.acunote-s...| 2|This Firefox Exte...|
 8| 1|https://github.co...|### Firefox Exten...| 1|acunote.acunote-s...| 3|%%NEWLINE Ready t...|
 9| 1|https://github.co...|### Greasemonkey ...| 1|acunote.acunote-s...| 3|%%NEWLINE To use ...|
 10| 1|https://github.co...|## Acunote: Power...| 1|acunote.acunote-s...| 2|%%NEWLINE <a href...|
 11| 1|https://github.co...| ## Contributing| 0|acunote.acunote-s...| 2|Acunote Shortcuts...|
 12| 2|https://github.co...|# TextMate bundle...| 1|akitaonrails.ruby...| 1|%%NEWLINE To inst...|
 13| 2|https://github.co...|## Rails 3.0 cont...| 0|akitaonrails.ruby...| 2|%%NEWLINE * Kamil...|
 14| 2|https://github.co...|## Rails 2.0 feat...| 0|akitaonrails.ruby...| 2|%%NEWLINE * Snipp...|
 15| 3|https://github.co...|# Introduction to...| 0|amueller.introduc...| 1|[![Binder](https:...|
 16| 3|https://github.co...| ## Errata| 1|amueller.introduc...| 2|Please note that ...|
 17| 3|https://github.co...| ## Setup| 1|amueller.introduc...| 2|%%NEWLINE To run ...|
 18| 3|https://github.co...|### Installing pa...| 1|amueller.introduc...| 3|If you already ha...|
 19| 3|https://github.co...|### Installing pa...| 1|amueller.introduc...| 3|If you already ha...|
 20| 3|https://github.co...|### Downloading E...| 1|amueller.introduc...| 3|For the text proc...|
+----------+-------+--------------------+--------------------+------------+--------------------+-------------+--------------------+
only showing top 20 rows

### Original Dataset (from paper)

Using the csv files available from the paper authors aswell as the readme files the following dataset was generated. The section_code reflects the papers  original labeling scheme.

In [0]:
df_original = sqlContext.sql("SELECT * FROM original_sections_csv")
df_original.show()

+----------+-------+--------------------+----------------------+------------+--------------------+-------------+---------------------------------+
section_id|file_id| url| heading_markdown|section_code| local_readme_file|heading_level| section_contents|
+----------+-------+--------------------+----------------------+------------+--------------------+-------------+---------------------------------+
 1| 1|https://github.co...| # [Easy WeChat](h...| -|xiaobai557.wechat.md| 1| %%NEWLINE 可能是目前最优...|
 2| 1|https://github.co...| ## 特点| -|xiaobai557.wechat.md| 2| %%NEWLINE - 命名不那...|
 3| 1|https://github.co...| ## 环境要求| -|xiaobai557.wechat.md| 2| %%NEWLINE 1. PHP ...|
 4| 1|https://github.co...| ## 安装| -|xiaobai557.wechat.md| 2| %%NEWLINE ```shel...|
 5| 1|https://github.co...| ## 使用| -|xiaobai557.wechat.md| 2| %%NEWLINE 基本使用（以服...|
 6| 1|https://github.co...| ## 文档| -|xiaobai557.wechat.md| 2| %%NEWLINE [http:/...|
 7| 1|https://github.co...| ## 框架集成| -|xiaobai557.wechat.md| 2| %%NEWLINE [Larave...|
 8| 1|https://github.co...| ## 贡献代码| -|xiaobai557.wechat.md| 2| %%NEWLINE [贡献指南](...|
 9| 1|https://github.co...| ## License| -|xiaobai557.wechat.md| 2| %%NEWLINE MIT%%NE...|
 10| 2|https://github.co...| # ThinkPHP 5.0| -|dungenessbin.thin...| 1| ===============%%...|
 11| 2|https://github.co...| ## 目录结构| -|dungenessbin.thin...| 2| %%NEWLINE 初始的目录结构...|
 12| 2|https://github.co...| ## 命名规范| -|dungenessbin.thin...| 2| %%NEWLINE `ThinkP...|
 13| 2|https://github.co...| ### 目录和文件| -|dungenessbin.thin...| 3| %%NEWLINE * 目录不...|
 14| 2|https://github.co...|### 函数和类、属性命名| -|dungenessbin.thin...| 3|* 类的命名采用驼峰法，并且首...|
 15| 2|https://github.co...| ### 常量和配置| -|dungenessbin.thin...| 3|* 常量以大写字母和下划线命名...|
 16| 2|https://github.co...| ### 数据表和字段| -|dungenessbin.thin...| 3|* 数据表和字段采用小写加下划...|
 17| 2|https://github.co...| ## 参与开发| -|dungenessbin.thin...| 2| 请参阅 [ThinkPHP5 核心...|
 18| 2|https://github.co...| ## 版权信息| -|dungenessbin.thin...| 2| %%NEWLINE ThinkPH...|
 19| 3|https://github.co...| # SYMON - A 6502 ...| 14| sethm.symon.md| 1| =================...|
 20| 3|https://github.co...| ## 1.0 About| 1246| sethm.symon.md| 2| %%NEWLINE Symon i...|
+----------+-------+--------------------+----------------------+------------+--------------------+-------------+---------------------------------+
only showing top 20 rows

#### Section Code convsersion

The section_code from the original dataset must be converted to 1 - how and 0 - not how.

In [0]:
# dropping undefined sections
df_original = df_original.where(df_original['section_code'] != '-')
df_original.show()

+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
section_id|file_id| url| heading_markdown|section_code|local_readme_file|heading_level| section_contents|
+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
 19| 3|https://github.co...|# SYMON - A 6502 ...| 14| sethm.symon.md| 1|=================...|
 20| 3|https://github.co...| ## 1.0 About| 1246| sethm.symon.md| 2|%%NEWLINE Symon i...|
 21| 3|https://github.co...| ## 2.0 Requirements| 3| sethm.symon.md| 2|%%NEWLINE - Jav...|
 22| 3|https://github.co...| ## 3.0 Features| 1| sethm.symon.md| 2|%%NEWLINE Symon c...|
 23| 3|https://github.co...| ### 3.1 Memory Maps| 1| sethm.symon.md| 3| %%NEWLINE|
 24| 3|https://github.co...|#### 3.1.1 Symon ...| 1| sethm.symon.md| 4|%%NEWLINE - `$0...|
 25| 3|https://github.co...|#### 3.1.2 MULTIC...| 1| sethm.symon.md| 4|%%NEWLINE - `$0...|
 26| 3|https://github.co...|### 3.1.3 Simple ...| 1| sethm.symon.md| 3|%%NEWLINE - `$0...|
 27| 3|https://github.co...|### 3.2 Serial Co...| 1| sethm.symon.md| 3|%%NEWLINE ![Seria...|
 28| 3|https://github.co...| ### 3.3 ROM Loading| 1| sethm.symon.md| 3|%%NEWLINE ![ROM L...|
 29| 3|https://github.co...|### 3.4 Memory Wi...| 1| sethm.symon.md| 3|%%NEWLINE ![Memor...|
 30| 3|https://github.co...| ### 3.5 Trace Log| 1| sethm.symon.md| 3|%%NEWLINE ![Trace...|
 31| 3|https://github.co...|### 3.6 Simulator...| 1| sethm.symon.md| 3|%%NEWLINE ![Speed...|
 32| 3|https://github.co...| ### 3.7 Breakpoints| 1| sethm.symon.md| 3|%%NEWLINE ![Break...|
 33| 3|https://github.co...|### 3.8 Experimen...| 16| sethm.symon.md| 3|%%NEWLINE ![Compo...|
 34| 3|https://github.co...|#### 3.8.1 Exampl...| 1| sethm.symon.md| 4|%%NEWLINE This pr...|
 35| 3|https://github.co...| ## 4.0 Usage| 3| sethm.symon.md| 2| %%NEWLINE|
 36| 3|https://github.co...| ### 4.1 Building| 3| sethm.symon.md| 3|%%NEWLINE To buil...|
 37| 3|https://github.co...| ### 4.2 ROM images| 3| sethm.symon.md| 3|%%NEWLINE The sim...|
 38| 3|https://github.co...|### 4.3 Loading A...| 3| sethm.symon.md| 3|%%NEWLINE In addi...|
+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
only showing top 20 rows

In [0]:
def update_section_code (section_code):
  section_code = str(section_code)
  if('3' in section_code):
    return 1
  else: return 0

In [0]:
from pyspark.sql.types import *
udf_update_section_code = udf(lambda x : update_section_code(x), IntegerType())

In [0]:
# updateing section codes to our format
df_original = df_original.withColumn("section_code", udf_update_section_code("section_code"))
df_original.show()

+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
section_id|file_id| url| heading_markdown|section_code|local_readme_file|heading_level| section_contents|
+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
 19| 3|https://github.co...|# SYMON - A 6502 ...| 0| sethm.symon.md| 1|=================...|
 20| 3|https://github.co...| ## 1.0 About| 0| sethm.symon.md| 2|%%NEWLINE Symon i...|
 21| 3|https://github.co...| ## 2.0 Requirements| 1| sethm.symon.md| 2|%%NEWLINE - Jav...|
 22| 3|https://github.co...| ## 3.0 Features| 0| sethm.symon.md| 2|%%NEWLINE Symon c...|
 23| 3|https://github.co...| ### 3.1 Memory Maps| 0| sethm.symon.md| 3| %%NEWLINE|
 24| 3|https://github.co...|#### 3.1.1 Symon ...| 0| sethm.symon.md| 4|%%NEWLINE - `$0...|
 25| 3|https://github.co...|#### 3.1.2 MULTIC...| 0| sethm.symon.md| 4|%%NEWLINE - `$0...|
 26| 3|https://github.co...|### 3.1.3 Simple ...| 0| sethm.symon.md| 3|%%NEWLINE - `$0...|
 27| 3|https://github.co...|### 3.2 Serial Co...| 0| sethm.symon.md| 3|%%NEWLINE ![Seria...|
 28| 3|https://github.co...| ### 3.3 ROM Loading| 0| sethm.symon.md| 3|%%NEWLINE ![ROM L...|
 29| 3|https://github.co...|### 3.4 Memory Wi...| 0| sethm.symon.md| 3|%%NEWLINE ![Memor...|
 30| 3|https://github.co...| ### 3.5 Trace Log| 0| sethm.symon.md| 3|%%NEWLINE ![Trace...|
 31| 3|https://github.co...|### 3.6 Simulator...| 0| sethm.symon.md| 3|%%NEWLINE ![Speed...|
 32| 3|https://github.co...| ### 3.7 Breakpoints| 0| sethm.symon.md| 3|%%NEWLINE ![Break...|
 33| 3|https://github.co...|### 3.8 Experimen...| 0| sethm.symon.md| 3|%%NEWLINE ![Compo...|
 34| 3|https://github.co...|#### 3.8.1 Exampl...| 0| sethm.symon.md| 4|%%NEWLINE This pr...|
 35| 3|https://github.co...| ## 4.0 Usage| 1| sethm.symon.md| 2| %%NEWLINE|
 36| 3|https://github.co...| ### 4.1 Building| 1| sethm.symon.md| 3|%%NEWLINE To buil...|
 37| 3|https://github.co...| ### 4.2 ROM images| 1| sethm.symon.md| 3|%%NEWLINE The sim...|
 38| 3|https://github.co...|### 4.3 Loading A...| 1| sethm.symon.md| 3|%%NEWLINE In addi...|
+----------+-------+--------------------+--------------------+------------+-----------------+-------------+--------------------+
only showing top 20 rows

### Comparing Original and New Data

#### Average Number of Sections per Readme

In [0]:
def avg_sections(df):
  num_sections  = df.count()
  num_readmes= df.drop_duplicates(subset=['url']).count()
  return num_sections/ num_readmes

In [0]:
avg_num_sections_new = avg_sections(df_new)
avg_num_sections_original = avg_sections(df_original)

print("The average number of sections per readme: \nOriginal Data: {:.2f} \nNew Data: {:.2f}".format(avg_num_sections_original, avg_num_sections_new))

The average number of sections per readme: 
Original Data: 10.75 
New Data: 10.54

#### Average Number of Terms per Section

In [0]:
 def avg_terms(df):
  num_sections  = df.count()
  df_contents = df.select('section_contents')
  return df_contents.rdd.map(lambda x: len(str(x).split())).reduce(lambda a,b: a + b)/num_sections

In [0]:
avg_terms_original = avg_terms(df_original)
avg_terms_new = avg_terms(df_new)
print("The average number of terms per section: \nOriginal Data: {:.2f} \nNew Data: {:.2f}".format(avg_terms_original, avg_terms_new))

The average number of terms per section: 
Original Data: 64.58 
New Data: 97.84

### Abstraction of heading and section contents

In [0]:
import re
def replace_NEWLINE(text):
  text = str(text)
  new_text = re.sub("%%NEWLINE", '\n', text)
  return new_text

In [0]:
udf_replace_NEWLINE = udf(lambda x : replace_NEWLINE(x), StringType())
df_original = df_original.withColumn("section_contents_newline",udf_replace_NEWLINE("section_contents"))
df_new = df_new.withColumn("section_contents_newline",udf_replace_NEWLINE("section_contents"))

In [0]:
import markdown2
from markdown2 import Markdown
from bs4 import BeautifulSoup, NavigableString, Tag
import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import re
import html2text

# Convert document to html
def convert_markdown_to_html(text_with_markdown):
     # Convert markdown to html, then parse text using BeautifulSoup
    # Use code-friendly extra syntax to avoid problem with strings such as factory_girl_rails changed into factory<em>girl</em>rails
    # and subsequently factorygirlrails after HTML tag removal
    markdowner = Markdown(extras=["code-friendly"])
    converted_md = markdowner.convert(text_with_markdown)
    return converted_md

def convert_html_to_markdown(html_text):
    # Instantiate HTML2Text object and set its body width to 0 to work around problem with it randomly adding "\n"
    # https://stackoverflow.com/questions/12839143/python-html2text-adds-random-n
    h = html2text.HTML2Text()
    h.body_width = 0
    return h.handle(html_text)

def abstract_text(markdown_text):
    markdown_text = str(markdown_text)
    # Perform code abstraction before HTML conversion to handle cases where there's embedded lines starting with # in the code
    # (which caused incorrect conversion)
    markdown_text = abstract_out_code_section(markdown_text)
    markdown_text = abstract_out_number(markdown_text, padding=True)
    
    html_text = convert_markdown_to_html(markdown_text)
    # Replace heading text first so any elements inside won't be changed to abstraction strings
    # html_text = replace_heading_content_with_plain_text(html_text)
    text_with_replaced_local_links = replace_local_links(html_text)
    abstracted_html_text = abstract_out_hyperlink_in_html(text_with_replaced_local_links, padding=True)
    abstracted_html_text = abstract_out_mailto_in_html(abstracted_html_text, padding=True)
    abstracted_html_text = abstract_out_image_in_html(abstracted_html_text, padding=True)
    # Commented out: Don't abstract inline code sections, since they may be used for names that may be useful
    # abstracted_html_text = abstract_out_code_in_html(abstracted_html_text, padding=True)
    abstracted_markdown_text = convert_html_to_markdown(abstracted_html_text)
    return abstracted_markdown_text

def abstract_out_code_section(markdown_text):
    # Remove Github-flavored markdown for syntax-highlighted code. E.g. ```obj-c .... ```
    # Add padding to prevent code section adjacent to a word from becoming @code_sectionword
    # Also handles case where there's space/tab before the ```
    # Use negative lookahead to avoid matching malformed block, e.g. `````` and  to avoid inline code followed by new line (e.g. ```xyz```
    # text
    # ```abc```)
    # from matching ```text```
    abstracted_text = re.sub(r'(?<!(```))(?<![a-z])```[ \t]{0,1}[a-z\.\-,\_]*[\s]*\n[\s\S]*?\n[\s]*(?<!(```))```',' @abstr_code_section ',markdown_text,0,re.DOTALL)
    return abstracted_text

# Abstract out number. Provide padding option to prevent number next to another character, e.g. X, from being turned into single word @abstr_numberX
def abstract_out_number(input_text, padding=False):
    # numbers_to_replace = re.findall(r'(\d+(?:\.\d+)?)',input_text)
    # Replace single set of consecutive digits at a time, instead of attempting to match "x.y" decimal format, to avoid problem with stuff like IP address

    if padding:
        output_text = re.sub("\d+", ' @abstr_number ', input_text)
    else:
        output_text = re.sub("\d+", '@abstr_number', input_text)

    return output_text

def replace_heading_content_with_plain_text(html_text):
    soup = BeautifulSoup(html_text, 'lxml')
    for i in [1,2,3,4,5,6,7]:
        for head in soup.findAll('h{0}'.format(i)):
            head_content_text = head.get_text(strip=False)
            head.string = head_content_text
    return str(soup)

# Changes local hyperlinks to their texts
def replace_local_links(html_text):
    soup = BeautifulSoup(html_text, 'lxml')
    for x in soup.find_all('a'):
        try:
            target_url = x['href']
            # External links and mailto are handled in another function
            match = re.match('^(http(s){0,1}:)|(www\.)', target_url)
            match2 = re.match('^mailto:', target_url)
            if match is None and match2 is None:
                x.replace_with(x.text)
        except KeyError as e:
            # Some documents contain anchors without href, e.g. microlv.prerender.md
            x.replace_with(x.text)
    return str(soup)

def abstract_out_hyperlink_in_html(html_text, padding=True):
    soup = BeautifulSoup(html_text, 'lxml')

    for link in soup.find_all('a'):
        if padding:
            link.replaceWith(" @abstr_hyperlink ")
        else:
            link.replaceWith("@abstr_hyperlink")
    return str(soup)

def abstract_out_mailto_in_html(html_text, padding=True):
    soup = BeautifulSoup(html_text, 'lxml')
    for mailto in soup.select('a[href^=mailto]'):
        if padding:
            mailto.replaceWith(" @abstr_mailto ")
        else:
            mailto.replaceWith("@abstr_mailto")
    return str(soup)

def abstract_out_image_in_html(html_text, padding=True):
    soup = BeautifulSoup(html_text, 'lxml')
    for c in soup.find_all('img'):
        if padding:
            c.replaceWith(" @abstr_image ")
        else:
            c.replaceWith("@abstr_image")
    return str(soup)
                    
def abstract_out_code_in_html(html_text, padding=True):
    soup = BeautifulSoup(html_text, 'lxml')
        
    for c in soup.find_all('code'):
        if padding:
            c.replaceWith(" @abstr_code_section ")
        else:
            c.replaceWith("@abstr_code_section")
    return str(soup)


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
udf_abstract_text = udf(lambda x : abstract_text(x), StringType())
df_original = df_original.withColumn("abstract_heading_markdown",udf_abstract_text("heading_markdown"))
df_original = df_original.withColumn("abstract_section_w_o_tag",udf_abstract_text("section_contents_newline"))
df_new = df_new.withColumn("abstract_heading_markdown",udf_abstract_text("heading_markdown"))
df_new = df_new.withColumn("abstract_section_w_o_tag",udf_abstract_text("section_contents_newline"))

In [0]:
def remove_underline(text):
  text = str(text)
  text_result = re.sub("=+","",text)
  return text_result.strip()
def remove_hashtag(text):
  text = str(text)
  text_result = re.sub("#+","",text)
  return text_result.strip()
udf_remove_underline = udf(lambda x : remove_underline(x), StringType())
udf_remove_hashtag = udf(lambda x : remove_hashtag(x), StringType())
df_new = df_new.withColumn("heading_text",udf_remove_hashtag("heading_markdown"))
df_new = df_new.withColumn("abstract_heading_text",udf_remove_hashtag("abstract_heading_markdown"))
df_new = df_new.withColumn("abstract_section_w_o_tag",udf_remove_underline("abstract_section_w_o_tag"))
df_original = df_original.withColumn("heading_text",udf_remove_hashtag("heading_markdown"))
df_original = df_original.withColumn("abstract_heading_text",udf_remove_hashtag("abstract_heading_markdown"))
df_original = df_original.withColumn("abstract_section_w_o_tag",udf_remove_underline("abstract_section_w_o_tag"))

In [0]:
df_original.select("heading_text","abstract_section_w_o_tag").show()

+--------------------+------------------------+
 heading_text|abstract_section_w_o_tag|
+--------------------+------------------------+
SYMON - A 6502 Sy...| **Version:** @abs...|
 1.0 About| Symon is a genera...|
 2.0 Requirements| * Java @abstr_num...|
 3.0 Features| Symon can simulat...|
 3.1 Memory Maps| |
3.1.1 Symon Memor...| * `$ @abstr_numbe...|
3.1.2 MULTICOMP M...| * `$ @abstr_numbe...|
3.1.3 Simple Memo...| * `$ @abstr_numbe...|
3.2 Serial Consol...| @abstr_image 

Th...|
 3.3 ROM Loading| @abstr_image 

Sy...|
 3.4 Memory Window| @abstr_image 

Me...|
 3.5 Trace Log| @abstr_image 

Th...|
3.6 Simulator Speeds| @abstr_image 

Si...|
 3.7 Breakpoints| @abstr_image 

Br...|
3.8 Experimental ...| @abstr_image 

Th...|
3.8.1 Example BAS...| This program will...|
 4.0 Usage| |
 4.1 Building| To build Symon wi...|
 4.2 ROM images| The simulator req...|
4.3 Loading A Pro...| In addition to RO...|
+--------------------+------------------------+
only showing top 20 rows

In [0]:
df_new.select("heading_text","abstract_section_w_o_tag").show()

+--------------------+------------------------+
 heading_text|abstract_section_w_o_tag|
+--------------------+------------------------+
Acunote's JavaScr...| |
 Overview| Acunote Shortcuts...|
 Try It| Firefox extension...|
 Features| * Simple to use: ...|
Four Simple Steps...| @abstr_number . @...|
Four Simple Steps...| Rails helper supp...|
News.YC, Reddit, ...| This Firefox Exte...|
 Firefox Extension| Ready to use. Jus...|
 Greasemonkey Script| To use Greasemonk...|
Acunote: Powered ...| Acunote, our proj...|
 Contributing| Acunote Shortcuts...|
TextMate bundle f...| To install with G...|
Rails 3.0 contrib...| * Kamil Kukura (r...|
Rails 2.0 feature...| * Snippets/Comman...|
Introduction to M...| @abstr_hyperlink ...|
 Errata| Please note that ...|
 Setup| To run the code, ...|
Installing packag...| If you already ha...|
Installing packag...| If you already ha...|
Downloading Engli...| For the text proc...|
+--------------------+------------------------+
only showing top 20 rows

#### Average Number of Terms per Section post Abstraction

In [0]:
 def avg_terms_abstracted(df):
  num_sections  = df.count()
  df_contents = df.select('abstract_section_w_o_tag')
  return df_contents.rdd.map(lambda x: len(str(x).split())).reduce(lambda a,b: a + b)/num_sections

In [0]:
avg_terms_new_abstract = avg_terms_abstracted(df_new)
avg_terms_original_abstract = avg_terms_abstracted(df_original)

print("The average number of terms per section: \nOriginal Data: {:.2f} \nNew Data: {:.2f}".format(avg_terms_original_abstract, avg_terms_new_abstract))

The average number of terms per section: 
Original Data: 54.25 
New Data: 92.90

### Creating Single DF

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql import functions as sf

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [0]:
df_combined = unionAll(df_original, df_new)

In [0]:
df_combined = df_combined.withColumn('heading_section_combined', sf.concat(sf.col('abstract_heading_markdown'),sf.lit(' '), sf.col("abstract_section_w_o_tag")))
df_original = df_original.withColumn('heading_section_combined', sf.concat(sf.col('abstract_heading_markdown'),sf.lit(' '), sf.col("abstract_section_w_o_tag")))

In [0]:
df_combined.select('heading_section_combined').show()

+------------------------+
heading_section_combined|
+------------------------+
 # SYMON - A @abst...|
 ## @abstr_number ...|
 ## @abstr_number ...|
 ## @abstr_number ...|
 ### @abstr_number...|
 #### @abstr_numbe...|
 #### @abstr_numbe...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
 #### @abstr_numbe...|
 ## @abstr_number ...|
 ### @abstr_number...|
 ### @abstr_number...|
 ### @abstr_number...|
+------------------------+
only showing top 20 rows

### Generating Features

In [0]:
from pyspark.ml.feature import IDF, Tokenizer, CountVectorizer, Binarizer
import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.

In [0]:
# Extracting statistical features with tokenization, removal of tokens, and stemming. 

def remove_stop_words(tokens):
  tokens_new = []
  for s in tokens:
    if s in stopWords:
      continue
    else:
      tokens_new.append(s)

  return tokens_new

def remove_abstracts(tokens):
  tokens_new = []
  for s in tokens:
    if "@" in s:
      continue
      
    #Optional to remove elif to remove all abstract tags
    #elif "_" in s:
      #continue
    tokens_new.append(s)
  return tokens_new

def stem_tokens(tokens):
  stemmer = PorterStemmer()
  stemmed = []
  for item in tokens:
    stemmed.append(stemmer.stem(item))
  return stemmed

def tokenize(text):
  tokens = word_tokenize(text)
  return tokens
  
def extract_statistical_features(col):
  tokens = tokenize(col)
  tokens = remove_stop_words(tokens)
  tokens = remove_abstracts(tokens)
  tokens = stem_tokens(tokens)
  
  return tokens


# UDF Method for Extraction
udf_extract_statistical_features = udf(lambda x:  extract_statistical_features(x), ArrayType(StringType()))


# Method to vectorize and IDF on extracted features
def vectorize_features(df_in, col_in, col_out):

  vecs = CountVectorizer(inputCol=col_in, outputCol="vector_features")
  model_vec = vecs.fit(df_in)
  featurizedData = model_vec.transform(df_in)

  idf = IDF(inputCol="vector_features", outputCol="idf_features")
  idfModel = idf.fit(featurizedData)
  rescaledData = idfModel.transform(featurizedData)
  
  binarizer = Binarizer(inputCol="idf_features", outputCol=col_out)

  binarizedDataFrame = binarizer.transform(rescaledData)
  
  return binarizedDataFrame

In [0]:
# All body and header heuristics functions

def heur_c_k_001(input_text):
    if ('report bugs' in input_text.lower()) or ('reporting bugs' in input_text.lower()):
        return 1
    else:
        return 0

def heur_c_k_002(input_text):
    if ' is a ' in input_text.lower():
        return 1
    else:
        return 0

def heur_c_k_003(input_text):
    if '@abstr_code_section' in input_text.lower():
        return 1
    else:
        return 0 
    
def heur_c_k_004(input_text):
    if 'attempts to' in input_text.lower():
        return 1
    else:
        return 0 

def heur_c_k_005(input_text):
    if 'inspired by' in input_text.lower():
        return 1
    else:
        return 0 

def heur_c_k_006(input_text):
    if 'install ' in input_text.lower():
        return 1
    else:
        return 0 

def heur_c_k_007(input_text):
    if 'reasons' in input_text.lower():
        return 1
    else:
        return 0

# Do not use lower() because we want to capture "Added" at beginning of sentence
def heur_c_k_008(input_text):
    if 'Added ' in input_text:
        return 1
    else:
        return 0

def heur_c_k_009(input_text):
    if 'copyright' in input_text.lower():
        return 1
    else:
        return 0

def heur_c_k_010(input_text):
    if '@abstr_mailto' in input_text.lower():
        return 1
    else:
        return 0

def heur_c_k_011(input_text):
    if 'you can ' in input_text.lower():
        return 1
    else:
        return 0

# Check if the text comprises solely of @abstr_code_section
def heur_c_k_012(input_text):
    if '@abstr_code_section' == input_text.lower().strip():
        return 1
    else:
        return 0

def heur_c_k_013(input_text):
    if 'About' in input_text:
        return 1
    else:
        return 0
    
def heur_c_k_014(input_text):
    if 'be sure to' in input_text.lower():
        return 1
    else:
        return 0    
    
def heur_c_k_015(input_text):
    if 'Download' in input_text:
        return 1
    else:
        return 0    
    
def heur_c_k_016(input_text):
    if 'overview' in input_text.lower():
        return 1
    else:
        return 0    
    
def heur_c_k_017(input_text):
    if 'get started' in input_text.lower():
        return 1
    else:
        return 0    
    
def heur_c_k_018(input_text):
    if 'reasons' in input_text.lower():
        return 1
    else:
        return 0   
    
def heur_c_k_019(input_text):
    if 'dependenc' in input_text.lower():
        return 1
    else:
        return 0   
    
def heur_c_k_020(input_text):
    if 'rerun' in input_text.lower():
        return 1
    else:
        return 0   
    
def heur_c_k_021(input_text):
    if 'you''ll be able' in input_text.lower():
        return 1
    else:
        return 0   
    
def heur_c_k_022(input_text):
    if 'you must' in input_text.lower():
        return 1
    else:
        return 0   
    
def heur_c_k_023(input_text):
    if 'previous version' in input_text.lower():
        return 1
    else:
        return 0   

def heur_h_k_001(input_text):
    if 'configur' in input_text.lower():
        return 1
    else:
        return 0 
     
def heur_h_k_002(input_text):
    if 'what' in input_text.lower():
        return 1
    else:
        return 0 
     
def heur_h_k_003(input_text):
    if 'why' in input_text.lower():
        return 1
    else:
        return 0   

def heur_h_k_004(input_text):
    if 'approach' in input_text.lower():
        return 1
    else:
        return 0   

def heur_h_k_005(input_text):
    if 'bugs' in input_text.lower():
        return 1
    else:
        return 0  
    
def heur_h_k_006(input_text):
    if 'contrib' in input_text.lower():
        return 1
    else:
        return 0         
    
def heur_h_k_007(input_text):
    if 'credit' in input_text.lower():
        return 1
    else:
        return 0            
    
def heur_h_k_008(input_text):
    if 'feature' in input_text.lower():
        return 1
    else:
        return 0              
    
def heur_h_k_009(input_text):
    if 'install' in input_text.lower():
        return 1
    else:
        return 0               
    
def heur_h_k_010(input_text):
    if 'intro' in input_text.lower():
        return 1
    else:
        return 0                 
    
def heur_h_k_011(input_text):
    if 'licen' in input_text.lower():
        return 1
    else:
        return 0             
    
def heur_h_k_012(input_text):
    if 'objective' in input_text.lower():
        return 1
    else:
        return 0      
    
def heur_h_k_013(input_text):
    if 'request' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_014(input_text):
    if 'requirement' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_015(input_text):
    if 'resource' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_016(input_text):
    if 'setting' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_017(input_text):
    if 'setup' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_018(input_text):
    if 'started' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_019(input_text):
    if 'usage' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_020(input_text):
    if 'version' in input_text.lower():
        return 1
    else:
        return 0    
    
def heur_h_k_021(input_text):
    if 'welcome' in input_text.lower():
        return 1
    else:
        return 0    
    
def heur_h_k_022(input_text):
    if 'what is' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_023(input_text):
    if 'overview' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_024(input_text):
    if 'basic' in input_text.lower():
        return 1
    else:
        return 0      
    
def heur_h_k_025(input_text):
    if 'roadmap' in input_text.lower():
        return 1
    else:
        return 0      
    
def heur_h_k_026(input_text):
    if 'todo' in input_text.lower():
        return 1
    else:
        return 0        
    
def heur_h_k_027(input_text):
    if 'example' in input_text.lower():
        return 1
    else:
        return 0         
    
def heur_h_k_028(input_text):
    if 'about' in input_text.lower():
        return 1
    else:
        return 0     
    
def heur_h_k_029(input_text):
    if 'reference' in input_text.lower():
        return 1
    else:
        return 0   
      
# See whether text comprises entirely of ASCII characters    
def heur_c_s_001(input_text):
    try:
        input_text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return 0
    else:
        return 1   
    

In [0]:
# Body text heuristics
udf_heur_c_k_001 = udf(lambda x:  heur_c_k_001(x), IntegerType())
udf_heur_c_k_002 = udf(lambda x:  heur_c_k_002(x), IntegerType())
udf_heur_c_k_003 = udf(lambda x:  heur_c_k_003(x), IntegerType())
udf_heur_c_k_004 = udf(lambda x:  heur_c_k_004(x), IntegerType())
udf_heur_c_k_005 = udf(lambda x:  heur_c_k_005(x), IntegerType())
udf_heur_c_k_006 = udf(lambda x:  heur_c_k_006(x), IntegerType())
udf_heur_c_k_007 = udf(lambda x:  heur_c_k_007(x), IntegerType())
udf_heur_c_k_008 = udf(lambda x:  heur_c_k_008(x), IntegerType())
udf_heur_c_k_009 = udf(lambda x:  heur_c_k_009(x), IntegerType())
udf_heur_c_k_010 = udf(lambda x:  heur_c_k_010(x), IntegerType())
udf_heur_c_k_011 = udf(lambda x:  heur_c_k_011(x), IntegerType())
udf_heur_c_k_012 = udf(lambda x:  heur_c_k_012(x), IntegerType())
udf_heur_c_k_013 = udf(lambda x:  heur_c_k_013(x), IntegerType())
udf_heur_c_k_014 = udf(lambda x:  heur_c_k_014(x), IntegerType())
udf_heur_c_k_015 = udf(lambda x:  heur_c_k_015(x), IntegerType())
udf_heur_c_k_016 = udf(lambda x:  heur_c_k_016(x), IntegerType())
udf_heur_c_k_017 = udf(lambda x:  heur_c_k_017(x), IntegerType())
udf_heur_c_k_018 = udf(lambda x:  heur_c_k_018(x), IntegerType())
udf_heur_c_k_019 = udf(lambda x:  heur_c_k_019(x), IntegerType())
udf_heur_c_k_020 = udf(lambda x:  heur_c_k_020(x), IntegerType())
udf_heur_c_k_021 = udf(lambda x:  heur_c_k_021(x), IntegerType())
udf_heur_c_k_022 = udf(lambda x:  heur_c_k_022(x), IntegerType())
udf_heur_c_k_023 = udf(lambda x:  heur_c_k_023(x), IntegerType())
udf_heur_c_s_001 = udf(lambda x:  heur_c_s_001(x), IntegerType())


def get_body_heuristics(df_in, col_in):
  df_in = df_in.withColumn("heur_c_k_001", udf_heur_c_k_001(col_in))
  df_in = df_in.withColumn("heur_c_k_002", udf_heur_c_k_002(col_in))
  df_in = df_in.withColumn("heur_c_k_003", udf_heur_c_k_003(col_in))
  df_in = df_in.withColumn("heur_c_k_004", udf_heur_c_k_004(col_in))
  df_in = df_in.withColumn("heur_c_k_005", udf_heur_c_k_005(col_in))
  df_in = df_in.withColumn("heur_c_k_006", udf_heur_c_k_006(col_in))
  df_in = df_in.withColumn("heur_c_k_007", udf_heur_c_k_007(col_in))
  df_in = df_in.withColumn("heur_c_k_008", udf_heur_c_k_008(col_in))
  df_in = df_in.withColumn("heur_c_k_009", udf_heur_c_k_009(col_in))
  df_in = df_in.withColumn("heur_c_k_010", udf_heur_c_k_010(col_in))
  df_in = df_in.withColumn("heur_c_k_011", udf_heur_c_k_011(col_in))
  df_in = df_in.withColumn("heur_c_k_012", udf_heur_c_k_012(col_in))
  df_in = df_in.withColumn("heur_c_k_013", udf_heur_c_k_013(col_in))
  df_in = df_in.withColumn("heur_c_k_014", udf_heur_c_k_014(col_in))
  df_in = df_in.withColumn("heur_c_k_015", udf_heur_c_k_015(col_in))
  df_in = df_in.withColumn("heur_c_k_016", udf_heur_c_k_016(col_in))
  df_in = df_in.withColumn("heur_c_k_017", udf_heur_c_k_017(col_in))
  df_in = df_in.withColumn("heur_c_k_018", udf_heur_c_k_018(col_in))
  df_in = df_in.withColumn("heur_c_k_019", udf_heur_c_k_019(col_in))
  df_in = df_in.withColumn("heur_c_k_020", udf_heur_c_k_020(col_in))
  df_in = df_in.withColumn("heur_c_k_021", udf_heur_c_k_021(col_in))
  df_in = df_in.withColumn("heur_c_k_022", udf_heur_c_k_022(col_in))
  df_in = df_in.withColumn("heur_c_k_023", udf_heur_c_k_023(col_in))
  df_in = df_in.withColumn("heur_c_s_001", udf_heur_c_s_001(col_in))

  return df_in

In [0]:
# Header heuristics
udf_heur_h_k_001 = udf(lambda x:  heur_h_k_001(x), IntegerType())
udf_heur_h_k_002 = udf(lambda x:  heur_h_k_002(x), IntegerType())
udf_heur_h_k_003 = udf(lambda x:  heur_h_k_003(x), IntegerType())
udf_heur_h_k_004 = udf(lambda x:  heur_h_k_004(x), IntegerType())
udf_heur_h_k_005 = udf(lambda x:  heur_h_k_005(x), IntegerType())
udf_heur_h_k_006 = udf(lambda x:  heur_h_k_006(x), IntegerType())
udf_heur_h_k_007 = udf(lambda x:  heur_h_k_007(x), IntegerType())
udf_heur_h_k_008 = udf(lambda x:  heur_h_k_008(x), IntegerType())
udf_heur_h_k_009 = udf(lambda x:  heur_h_k_009(x), IntegerType())
udf_heur_h_k_010 = udf(lambda x:  heur_h_k_010(x), IntegerType())
udf_heur_h_k_011 = udf(lambda x:  heur_h_k_011(x), IntegerType())
udf_heur_h_k_012 = udf(lambda x:  heur_h_k_012(x), IntegerType())
udf_heur_h_k_013 = udf(lambda x:  heur_h_k_013(x), IntegerType())
udf_heur_h_k_014 = udf(lambda x:  heur_h_k_014(x), IntegerType())
udf_heur_h_k_015 = udf(lambda x:  heur_h_k_015(x), IntegerType())
udf_heur_h_k_016 = udf(lambda x:  heur_h_k_016(x), IntegerType())
udf_heur_h_k_017 = udf(lambda x:  heur_h_k_017(x), IntegerType())
udf_heur_h_k_018 = udf(lambda x:  heur_h_k_018(x), IntegerType())
udf_heur_h_k_019 = udf(lambda x:  heur_h_k_019(x), IntegerType())
udf_heur_h_k_020 = udf(lambda x:  heur_h_k_020(x), IntegerType())
udf_heur_h_k_021 = udf(lambda x:  heur_h_k_021(x), IntegerType())
udf_heur_h_k_022 = udf(lambda x:  heur_h_k_022(x), IntegerType())
udf_heur_h_k_023 = udf(lambda x:  heur_h_k_023(x), IntegerType())
udf_heur_h_k_024 = udf(lambda x:  heur_h_k_024(x), IntegerType())
udf_heur_h_k_025 = udf(lambda x:  heur_h_k_025(x), IntegerType())
udf_heur_h_k_026 = udf(lambda x:  heur_h_k_026(x), IntegerType())
udf_heur_h_k_027 = udf(lambda x:  heur_h_k_027(x), IntegerType())
udf_heur_h_k_028 = udf(lambda x:  heur_h_k_028(x), IntegerType())
udf_heur_h_k_029 = udf(lambda x:  heur_h_k_029(x), IntegerType())

def get_header_heuristics(df_in, col_in):
  df_in = df_in.withColumn("heur_h_k_001", udf_heur_h_k_001(col_in))
  df_in = df_in.withColumn("heur_h_k_002", udf_heur_h_k_002(col_in))
  df_in = df_in.withColumn("heur_h_k_003", udf_heur_h_k_003(col_in))
  df_in = df_in.withColumn("heur_h_k_004", udf_heur_h_k_004(col_in))
  df_in = df_in.withColumn("heur_h_k_005", udf_heur_h_k_005(col_in))
  df_in = df_in.withColumn("heur_h_k_006", udf_heur_h_k_006(col_in))
  df_in = df_in.withColumn("heur_h_k_007", udf_heur_h_k_007(col_in))
  df_in = df_in.withColumn("heur_h_k_008", udf_heur_h_k_008(col_in))
  df_in = df_in.withColumn("heur_h_k_009", udf_heur_h_k_009(col_in))
  df_in = df_in.withColumn("heur_h_k_010", udf_heur_h_k_010(col_in))
  df_in = df_in.withColumn("heur_h_k_011", udf_heur_h_k_011(col_in))
  df_in = df_in.withColumn("heur_h_k_012", udf_heur_h_k_012(col_in))
  df_in = df_in.withColumn("heur_h_k_013", udf_heur_h_k_013(col_in))
  df_in = df_in.withColumn("heur_h_k_014", udf_heur_h_k_014(col_in))
  df_in = df_in.withColumn("heur_h_k_015", udf_heur_h_k_015(col_in))
  df_in = df_in.withColumn("heur_h_k_016", udf_heur_h_k_016(col_in))
  df_in = df_in.withColumn("heur_h_k_017", udf_heur_h_k_017(col_in))
  df_in = df_in.withColumn("heur_h_k_018", udf_heur_h_k_018(col_in))
  df_in = df_in.withColumn("heur_h_k_019", udf_heur_h_k_019(col_in))
  df_in = df_in.withColumn("heur_h_k_020", udf_heur_h_k_020(col_in))
  df_in = df_in.withColumn("heur_h_k_021", udf_heur_h_k_021(col_in))
  df_in = df_in.withColumn("heur_h_k_022", udf_heur_h_k_022(col_in))
  df_in = df_in.withColumn("heur_h_k_023", udf_heur_h_k_023(col_in))
  df_in = df_in.withColumn("heur_h_k_024", udf_heur_h_k_024(col_in))
  df_in = df_in.withColumn("heur_h_k_025", udf_heur_h_k_025(col_in))
  df_in = df_in.withColumn("heur_h_k_026", udf_heur_h_k_026(col_in))
  df_in = df_in.withColumn("heur_h_k_027", udf_heur_h_k_027(col_in))
  df_in = df_in.withColumn("heur_h_k_028", udf_heur_h_k_028(col_in))
  df_in = df_in.withColumn("heur_h_k_029", udf_heur_h_k_029(col_in))
  
  return df_in

In [0]:
# Special heuristic cases
import re

# Returns 1 if a word in heading text is in repo name, or the other way around
def heur_h_c_002(heading_text, repo_url):
    heading_words = heading_text.lower().split(' ')
    repo_name = re.sub(r"http(s)*:\/\/www\.github\.com\/.+\/",'',repo_url)
    repo_name = repo_name.replace('.',' ').replace('-',' ')
    repo_name_words = repo_name.lower().split(' ')
    match = [val for val in heading_words if val in repo_name_words]
    if len(match) >0:
        return 1
    else:
        return 0

udf_heur_h_c_002 = udf(lambda x,y:  heur_h_c_002(x,y), IntegerType())

def get_special_heuristics(df_in, col_in, repo_col):
  df_in = df_in.withColumn("heur_h_c_002", udf_heur_h_c_002(col_in, repo_col))
  
  return df_in

In [0]:
# Getting Statistical features
df_combined_tokens = df_combined.withColumn("raw_stat_features", udf_extract_statistical_features("heading_section_combined"))
df_combined_vectorized =  vectorize_features(df_combined_tokens, "raw_stat_features", "total_vector_features")

df_original_tokens = df_original.withColumn("raw_stat_features", udf_extract_statistical_features("heading_section_combined"))
df_original_vectorized =  vectorize_features(df_original_tokens, "raw_stat_features", "total_vector_features")

In [0]:
# Getting Heuristic features
df_combined_heuristics = get_body_heuristics(df_combined_vectorized, "abstract_section_w_o_tag")
df_combined_heuristics = get_header_heuristics(df_combined_heuristics, "abstract_heading_markdown")
df_combined_heuristics = get_special_heuristics(df_combined_heuristics, "abstract_heading_markdown", "url")

df_original_heuristics = get_body_heuristics(df_original_vectorized, "abstract_section_w_o_tag")
df_original_heuristics = get_header_heuristics(df_original_heuristics, "abstract_heading_markdown")
df_original_heuristics = get_special_heuristics(df_original_heuristics, "abstract_heading_markdown", "url")

In [0]:
# Assembling features into one column
from pyspark.ml.feature import VectorAssembler

col_list = ["total_vector_features", "heur_h_k_001", "heur_h_k_002", "heur_h_k_003", "heur_h_k_004", "heur_h_k_005", "heur_h_k_006", "heur_h_k_007", "heur_h_k_008", "heur_h_k_009", "heur_h_k_010", "heur_h_k_011", "heur_h_k_012", "heur_h_k_013", "heur_h_k_014", "heur_h_k_015", "heur_h_k_016", "heur_h_k_017", "heur_h_k_018", "heur_h_k_019", "heur_h_k_020", "heur_h_k_021", "heur_h_k_022", "heur_h_k_023", "heur_h_k_024", "heur_h_k_025", "heur_h_k_026", "heur_h_k_027", "heur_h_k_028", "heur_h_k_029", "heur_c_k_001", "heur_c_k_002", "heur_c_k_003", "heur_c_k_004", "heur_c_k_005", "heur_c_k_006", "heur_c_k_007", "heur_c_k_008", "heur_c_k_009", "heur_c_k_010", "heur_c_k_011", "heur_c_k_012", "heur_c_k_013", "heur_c_k_014", "heur_c_k_015", "heur_c_k_016", "heur_c_k_017", "heur_c_k_018", "heur_c_k_019", "heur_c_k_020", "heur_c_k_021", "heur_c_k_022", "heur_c_k_023", "heur_c_s_001", "heur_h_c_002"]

assembler = VectorAssembler(inputCols=col_list, outputCol="features")

df_combined_ml = assembler.transform(df_combined_heuristics)
df_original_ml = assembler.transform(df_original_heuristics)

### Train Test Splits

In [0]:
# Splitting Training and Test Data
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.mllib.util import MLUtils

#df_combined_ml = df_combined_ml.select("features", "section_code")
df_combined_ml = df_combined_ml.withColumnRenamed("section_code", "label")
#df_ml = MLUtils.convertVectorColumnsFromML(df_ml, "features")

#df_original_ml = df_original_ml.select("features", "section_code")
df_original_ml = df_original_ml.withColumnRenamed("section_code", "label")

combined_train, combined_test = df_combined_ml.randomSplit([0.9, 0.1], seed=12345)
original_train, original_test = df_original_ml.randomSplit([0.9, 0.1], seed=12345)

In [0]:
combined_train.cache()
combined_test.cache()
original_train.cache()
original_test.cache()

Out[42]: DataFrame[section_id: int, file_id: int, url: string, heading_markdown: string, label: int, local_readme_file: string, heading_level: int, section_contents: string, section_contents_newline: string, abstract_heading_markdown: string, abstract_section_w_o_tag: string, heading_text: string, abstract_heading_text: string, heading_section_combined: string, raw_stat_features: array<string>, vector_features: vector, idf_features: vector, total_vector_features: vector, heur_c_k_001: int, heur_c_k_002: int, heur_c_k_003: int, heur_c_k_004: int, heur_c_k_005: int, heur_c_k_006: int, heur_c_k_007: int, heur_c_k_008: int, heur_c_k_009: int, heur_c_k_010: int, heur_c_k_011: int, heur_c_k_012: int, heur_c_k_013: int, heur_c_k_014: int, heur_c_k_015: int, heur_c_k_016: int, heur_c_k_017: int, heur_c_k_018: int, heur_c_k_019: int, heur_c_k_020: int, heur_c_k_021: int, heur_c_k_022: int, heur_c_k_023: int, heur_c_s_001: int, heur_h_k_001: int, heur_h_k_002: int, heur_h_k_003: int, heur_h_k_004: int, heur_h_k_005: int, heur_h_k_006: int, heur_h_k_007: int, heur_h_k_008: int, heur_h_k_009: int, heur_h_k_010: int, heur_h_k_011: int, heur_h_k_012: int, heur_h_k_013: int, heur_h_k_014: int, heur_h_k_015: int, heur_h_k_016: int, heur_h_k_017: int, heur_h_k_018: int, heur_h_k_019: int, heur_h_k_020: int, heur_h_k_021: int, heur_h_k_022: int, heur_h_k_023: int, heur_h_k_024: int, heur_h_k_025: int, heur_h_k_026: int, heur_h_k_027: int, heur_h_k_028: int, heur_h_k_029: int, heur_h_c_002: int, features: vector]

### Training Models

#### LinearSVC

In [0]:
from pyspark.ml.classification import LinearSVC

lsvc_combined = LinearSVC()
lsvc_original = LinearSVC()

lsvc_combined_model = lsvc_combined.fit(combined_train.select("features", "label"))
lsvc_original_model = lsvc_original.fit(original_train.select("features", "label"))

#### Gradient Boosted Tree

In [0]:
from pyspark.ml.classification import GBTClassifier

gbtc_combined = GBTClassifier()
gbtc_original = GBTClassifier()

gbtc_combined_model = gbtc_combined.fit(combined_train.select("features", "label"))
gbtc_original_model = gbtc_original.fit(original_train.select("features", "label"))

### Predictions with Test Data

In [0]:
lsvc_combined_transform = lsvc_combined_model.transform(combined_test.select("features", "label"))
lsvc_original_transform = lsvc_original_model.transform(original_test.select("features", "label"))

In [0]:
gbtc_combined_transform = gbtc_combined_model.transform(combined_test.select("features", "label"))
gbtc_original_transform = gbtc_original_model.transform(original_test.select("features", "label"))

#### Classification Report

In [0]:
def falseNegatives(df):
  return df.where((df['label']==0) & (df['prediction']==1.0)).count()

def falsePositives(df):
  return df.where((df['label']==1) & (df['prediction']==0.0)).count()

def trueNegatives(df):
   return df.where((df['label'] == 0) & (df['prediction']== 0.0)).count()
  
def truePositives(df):
  return df.where((df['prediction'] == 1.0) & (df['label'] == 1)).count()

In [0]:
def classification_report(df):
  tp = truePositives(df)
  tn = trueNegatives(df)
  fn = falseNegatives(df)
  fp = falsePositives(df)

  accuracy = (tp + tn) / (tp + tn + fn + fp)
  recall = tp/(tp+fn)
  precision = tp/(tp+fp)
  f1score = (2*precision*recall)/(precision+recall)
  
  print("Summary Stats:")
  print("Accuracy = {:.2f}".format(accuracy))
  print("Precision = {:.2f}".format(precision))
  print("Recall = {:.2f}".format(recall))
  print("F1 Score = {:.2f}".format(f1score))
  print()

In [0]:
print("The Classification Report for LinearSVC on Combined Data:")
classification_report(lsvc_combined_transform)

print("The Classification Report for LinearSVC on Original Data:")
classification_report(lsvc_original_transform)

print("The Classification Report for Gradient Boosted Tree Classifier on Combined Data:")
classification_report(gbtc_combined_transform)

print("The Classification Report for Gradient Boosted Tree Classifier on Original Data:")
classification_report(gbtc_original_transform)

The Classification Report for LinearSVC on Combined Data:
Summary Stats:
Accuracy = 0.86
Precision = 0.91
Recall = 0.86
F1 Score = 0.89

The Classification Report for LinearSVC on Original Data:
Summary Stats:
Accuracy = 0.83
Precision = 0.89
Recall = 0.83
F1 Score = 0.86

The Classification Report for Gradient Boosted Tree Classifier on Combined Data:
Summary Stats:
Accuracy = 0.79
Precision = 0.83
Recall = 0.82
F1 Score = 0.82

The Classification Report for Gradient Boosted Tree Classifier on Original Data:
Summary Stats:
Accuracy = 0.79
Precision = 0.92
Recall = 0.77
F1 Score = 0.84

### Tuning Hyperparameters

In [0]:
from pyspark.ml.classification import LinearSVC
regParam = [0.001, 0.005, 0.01, 0.1]
for reg in regParam:
  lsvc_combined = LinearSVC(regParam=reg)
  lsvc_original = LinearSVC(regParam=reg)
  lsvc_combined_model = lsvc_combined.fit(combined_train.select("features", "label"))
  lsvc_original_model = lsvc_original.fit(original_train.select("features", "label"))
  lsvc_combined_transform = lsvc_combined_model.transform(combined_test.select("features", "label"))
  lsvc_original_transform = lsvc_original_model.transform(original_test.select("features", "label"))
  print("Regularization parameter: {}".format(reg))
  print("The Classification Report for LinearSVC on Combined Data:")
  classification_report(lsvc_combined_transform)
  print("The Classification Report for LinearSVC on Original Data:")
  classification_report(lsvc_original_transform)
  print("=========================================================")

Regularization parameter: 0.001
The Classification Report for LinearSVC on Combined Data:
Summary Stats:
Accuracy = 0.86
Precision = 0.92
Recall = 0.86
F1 Score = 0.89

The Classification Report for LinearSVC on Original Data:
Summary Stats:
Accuracy = 0.84
Precision = 0.90
Recall = 0.84
F1 Score = 0.87

=========================================================
Regularization parameter: 0.005
The Classification Report for LinearSVC on Combined Data:
Summary Stats:
Accuracy = 0.87
Precision = 0.92
Recall = 0.86
F1 Score = 0.89

The Classification Report for LinearSVC on Original Data:
Summary Stats:
Accuracy = 0.84
Precision = 0.90
Recall = 0.84
F1 Score = 0.87

=========================================================
Regularization parameter: 0.01
The Classification Report for LinearSVC on Combined Data:
Summary Stats:
Accuracy = 0.86
Precision = 0.92
Recall = 0.86
F1 Score = 0.89

The Classification Report for LinearSVC on Original Data:
Summary Stats:
Accuracy = 0.84
Precision = 0.91
Recall = 0.84
F1 Score = 0.87

=========================================================
Regularization parameter: 0.1
The Classification Report for LinearSVC on Combined Data:
Summary Stats:
Accuracy = 0.86
Precision = 0.92
Recall = 0.85
F1 Score = 0.88

The Classification Report for LinearSVC on Original Data:
Summary Stats:
Accuracy = 0.84
Precision = 0.91
Recall = 0.84
F1 Score = 0.87

=========================================================

In [0]:
from pyspark.ml.classification import GBTClassifier
maxDepths = [5, 6, 7, 8]
for depth in maxDepths:
  gbtc_combined = GBTClassifier(maxDepth=depth)
  gbtc_original = GBTClassifier(maxDepth=depth)
  gbtc_combined_model = gbtc_combined.fit(combined_train.select("features", "label"))
  gbtc_original_model = gbtc_original.fit(original_train.select("features", "label"))
  gbtc_combined_transform = gbtc_combined_model.transform(combined_test.select("features", "label"))
  gbtc_original_transform = gbtc_original_model.transform(original_test.select("features", "label"))
  print("Maximum decision tree depth: {}".format(depth))
  print("The Classification Report for Gradient Boosted Tree Classifier on Combined Data:")
  classification_report(gbtc_combined_transform)
  print("The Classification Report for Gradient Boosted Tree Classifier on Original Data:")
  classification_report(gbtc_original_transform)
  print("================================================================================")

Maximum decision tree depth: 5
The Classification Report for Gradient Boosted Tree Classifier on Combined Data:
Summary Stats:
Accuracy = 0.79
Precision = 0.83
Recall = 0.82
F1 Score = 0.82

The Classification Report for Gradient Boosted Tree Classifier on Original Data:
Summary Stats:
Accuracy = 0.79
Precision = 0.92
Recall = 0.77
F1 Score = 0.84

================================================================================
Maximum decision tree depth: 6
The Classification Report for Gradient Boosted Tree Classifier on Combined Data:
Summary Stats:
Accuracy = 0.79
Precision = 0.83
Recall = 0.82
F1 Score = 0.82

The Classification Report for Gradient Boosted Tree Classifier on Original Data:
Summary Stats:
Accuracy = 0.83
Precision = 0.84
Recall = 0.87
F1 Score = 0.85

================================================================================
Maximum decision tree depth: 7
The Classification Report for Gradient Boosted Tree Classifier on Combined Data:
Summary Stats:
Accuracy = 0.81
Precision = 0.83
Recall = 0.85
F1 Score = 0.84

The Classification Report for Gradient Boosted Tree Classifier on Original Data:
Summary Stats:
Accuracy = 0.82
Precision = 0.83
Recall = 0.87
F1 Score = 0.85

================================================================================
Maximum decision tree depth: 8
The Classification Report for Gradient Boosted Tree Classifier on Combined Data:
Summary Stats:
Accuracy = 0.81
Precision = 0.84
Recall = 0.84
F1 Score = 0.84

The Classification Report for Gradient Boosted Tree Classifier on Original Data:
Summary Stats:
Accuracy = 0.82
Precision = 0.82
Recall = 0.87
F1 Score = 0.84

================================================================================

### Misclassified Samples

In [0]:
# best performing model
from pyspark.ml.classification import LinearSVC
lsvc_combined = LinearSVC(regParam=0.005)
lsvc_combined_model = lsvc_combined.fit(combined_train.select("features", "label"))
lsvc_combined_transform = lsvc_combined_model.transform(df_combined_ml.select("features", "label"))

In [0]:
import pandas as pd
df_misclassified = pd.DataFrame()
df_misclassified['url'] = df_combined_ml.select("url").rdd.map(lambda x: x[0]).collect()
df_misclassified['Heading'] = df_combined_ml.select("heading_text").rdd.map(lambda x: x[0]).collect()
df_misclassified['Label'] = lsvc_combined_transform.select("label").rdd.map(lambda x: x[0]).collect()
df_misclassified['Predicted'] = lsvc_combined_transform.select("prediction").rdd.map(lambda x: x[0]).collect()
df_misclassified = df_misclassified[df_misclassified.Label != df_misclassified.Predicted]
df_misclassified

Out[61]:

,url,Heading,Label,Predicted
36,https://github.com/microlv/prerender,Middleware,0,1.0
98,https://github.com/LoMicah03/hello-world-ruby-...,Submitting Your Lab,1,0.0
255,https://github.com/owap/map-journal-storytelli...,Security,0,1.0
310,https://github.com/abparker/developer-exercise,Web Application Developer Exercise,0,1.0
357,https://github.com/cwohlman/delta,Utility,0,1.0
...,...,...,...,...
4694,https://github.com/ShiqiYu/libfacedetection,libfacedetection,0,1.0
4704,https://github.com/Skar0/fire-detection,Datasets,0,1.0
4717,https://github.com/Skar0/fire-detection,Results,0,1.0
4720,https://github.com/Skar0/fire-detection,Video examples,0,1.0


In [0]:
df_misclassified.to_csv('/dbfs/misclassified_results.csv')

display(df_misclassified)

url,Heading,Label,Predicted
https://github.com/microlv/prerender,Middleware,0,1.0
https://github.com/LoMicah03/hello-world-ruby-ruby-intro-000,Submitting Your Lab,1,0.0
https://github.com/owap/map-journal-storytelling-template-js,Security,0,1.0
https://github.com/abparker/developer-exercise,Web Application Developer Exercise,0,1.0
https://github.com/cwohlman/delta,Utility,0,1.0
https://github.com/cwohlman/delta,Operational Transform,0,1.0
https://github.com/cwohlman/delta,Documents,1,0.0
https://github.com/pombredanne/cassle,State,0,1.0
https://github.com/RLC22/butter-desktop,Screenshots,0,1.0
https://github.com/mcansh/simple-partials-reading-v-000,Simple Partials,0,1.0


In [0]:
def print_misclassification_percent(df, label):
  total = df.count()
  label_count = df.where(df['Misclassification'] == label).count()
  print("{:.2f}% of misslassifcations fall under label {}".format((label_count/total * 100), label))

In [0]:
df_misclassified_labeled = sqlContext.sql("SELECT * FROM misclassified_samples_csv")

In [0]:
for label in ["0","1","2","3"]:
  print_misclassification_percent(df_misclassified_labeled, label)
  print()

22.79% of misslassifcations fall under label 0

15.44% of misslassifcations fall under label 1

38.24% of misslassifcations fall under label 2

13.97% of misslassifcations fall under label 3